# HW1 - part 4
<a id=part4></a>

In [21]:
import sys, random,binascii,AES,jupyter_utils,random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, display,HTML
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150


This section will focus on the Diffusion principle and examine the extent to which a change in input or internal states affects the end result. The runs in the strain part will be performed on a single input block (16 characters)

In [22]:
plaintext = "From the Pope to"
print("Total Length:",len(plaintext))
display(Markdown(f'\n"**_{plaintext}_**"'))


Total Length: 16



"**_From the Pope to_**"

And we will try to encrypt the message, using an encryption key **"R is a Short key"**
 

In [23]:
key = "R is a Short key"
key = key.encode().hex()
print(f"'R is a Short key':\n 0x{key}")
aes_lib = AES.AES(int(key,base=16))

'R is a Short key':
 0x5220697320612053686f7274206b6579


## Question 4:
1. Run the AES algorithm, and save the intermediate results after each round (including the final result)
2. Randomly select a bit in the login message and change it from 0 -> 1 or 1-> 0 accordingly Repeat (1) and calculate how many bits changed their value as a result of the change of the single bit in each round, and the encrypted message (relative to the same situation when no errors were injected)
4. Repeat the experiment 5 times when each time you change one bit in relation to the message in (1) at the end of the runs, present a table

In [24]:
def run_aes_stages(plaintext):
    spec_enc_text = [""]*10
    for i in range(0, len(plaintext), 16):  
        ciper = aes_lib.encrypt_by_stage(int(plaintext[i:i+16].encode().hex(),base=16))
        for j,c in enumerate(ciper):
            spec_enc_text[j] += ('{:x}'.format(c)).zfill(32)
    return spec_enc_text
spec_enc_text = run_aes_stages(plaintext)
display(spec_enc_text)

['cb95e6a2a56a602164f8bdc338622f85',
 'cbf42cb3f27bbb6266e1e7abbc87199e',
 '5953b07e2a304dc27be50e6355176310',
 '203d5627e023761e3d86efd4ace38e38',
 'b874b055f6f6fc733e8c7952dbf7ea9c',
 '1dba47880ed16f3c72d966c2526347cc',
 '737bf4f4317604824733b3b13695b2cf',
 'd0d5082d7e23c65c2e2cd9304833f9f6',
 '54d92855bb1e7c8075c0eb813950e85d',
 '793ec12a27911757630e1fd3c4affd06']

For stage II and on, we wants' to create a convertion function

In [25]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8 ):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def count_flips_to_convert(a, b):

    diff = a ^ b

    # count number of ones in diff
    count = 0
    while diff:
        diff &= (diff - 1)
        count += 1
    return count

In [26]:
def flip_plaintext(plaintext):
    bitflip_p = tobits(plaintext)
    place = random.choice(range(len(bitflip_p)))
    bitflip_p[place] = int(1 - bitflip_p[place])
    bitflip_plaintext = frombits(bitflip_p)
    return bitflip_plaintext,place
bitflip_plaintext,place = flip_plaintext(plaintext)
print("The bit which was chosen to be replaced: ", place)
print("Result:",bitflip_plaintext)

The bit which was chosen to be replaced:  100
Result: From the Popm to


In [27]:
bitflip_spec_enc_text = run_aes_stages(bitflip_plaintext)
display(bitflip_spec_enc_text)

['cb95e6a2a56a602164f8bdc339efa209',
 '5d628d84dd0ae54df90b92de96920ca1',
 'd7c34364cc89d487b9c27f60d7e264cc',
 'b7af5ab0b00d84c8a40f47bc8d418f0b',
 'df3ed5a1ed016177da562e52f003c1b0',
 '1e9fd54e75a301a80857cdf507f7ecec',
 '93c9853e6ac5a61583a97643c1aab5fd',
 '448016b430f6dbe045459ddea91ea2e4',
 '6eec8cb1997c3694dfd40deb7d589634',
 'c65cff7e23631c6360374f3c2954e8e3']

In [28]:
def calulate_diffrence(textlist,flip_textlist):
    diff = []
    for i in range(len(flip_textlist)):
        p =  int(textlist[i],base=16)
        c =  int(flip_textlist[i],base=16)
        diff.append(count_flips_to_convert(p,c))
    return diff
diff = calulate_diffrence(bitflip_spec_enc_text,spec_enc_text)
display(diff)

[12, 71, 61, 54, 64, 62, 68, 65, 53, 66]

In [29]:
table_result = np.zeros((6,len(bitflip_spec_enc_text)))

for i in range(5):
    bfp,place = flip_plaintext(plaintext)
    bfp_stage = run_aes_stages(bfp)
    table_result[i] = calulate_diffrence(bfp_stage,spec_enc_text)


In [30]:
table_result[5] = np.mean(table_result[:5], axis = 0)
display(pd.DataFrame(table_result, columns=[f"Round {col+1}" for col in range(len(bitflip_spec_enc_text))]))

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8,Round 9,Round 10
0,10.0,59.0,65.0,61.0,56.0,73.0,59.0,66.0,65.0,60.0
1,12.0,69.0,63.0,63.0,64.0,63.0,61.0,75.0,57.0,63.0
2,17.0,64.0,60.0,66.0,55.0,65.0,57.0,68.0,66.0,60.0
3,71.0,60.0,59.0,60.0,66.0,65.0,66.0,58.0,56.0,64.0
4,18.0,50.0,65.0,69.0,65.0,59.0,56.0,66.0,67.0,71.0
5,25.6,60.4,62.4,63.8,61.2,65.0,59.8,66.6,62.2,63.6
